# Análise Exploratória de Dados SQLite para Treinamento de Redes Neurais

Este notebook realiza uma análise exploratória completa de dados armazenados em arquivos SQLite3, preparando-os para treinamento de modelos de aprendizado profundo.

## Objetivos:
- Conectar e explorar bases de dados SQLite3
- Analisar variáveis numéricas, categóricas e campos de texto
- Identificar problemas de qualidade de dados
- Preparar dados para modelos de redes neurais

## 1. Importação de Bibliotecas

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Bibliotecas importadas com sucesso!")

## 2. Configuração e Conexão ao Banco de Dados

In [ ]:
# Configurar o caminho para o arquivo SQLite
DB_PATH = r"c:\Users\bruno.silva\Codes\cnn-classificador-peticoes\seu_arquivo.db"  # Altere para o caminho do seu arquivo

# Verificar se o arquivo existe
if not Path(DB_PATH).exists():
    print(f"AVISO: Arquivo não encontrado em {DB_PATH}")
    print("Por favor, atualize o caminho do arquivo SQLite na variável DB_PATH")
else:
    print(f"Arquivo encontrado: {DB_PATH}")

In [ ]:
# Função para conectar ao banco de dados
def connect_to_db(db_path):
    try:
        conn = sqlite3.connect(db_path)
        print(f"Conexão estabelecida com sucesso!")
        return conn
    except sqlite3.Error as e:
        print(f"Erro ao conectar: {e}")
        return None

conn = connect_to_db(DB_PATH)

## 3. Exploração da Estrutura do Banco de Dados

In [ ]:
# Listar todas as tabelas disponíveis
def list_tables(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(query, conn)
    return tables

if conn:
    tables_df = list_tables(conn)
    print(f"Tabelas encontradas: {len(tables_df)}")
    print("\nLista de tabelas:")
    display(tables_df)

In [ ]:
# Explorar schema de cada tabela
def get_table_info(conn, table_name):
    query = f"PRAGMA table_info({table_name});"
    return pd.read_sql_query(query, conn)

if conn and not tables_df.empty:
    print("\n" + "="*80)
    print("ESTRUTURA DAS TABELAS")
    print("="*80)
    
    for table_name in tables_df['name']:
        print(f"\n📊 Tabela: {table_name}")
        schema = get_table_info(conn, table_name)
        display(schema)
        
        # Contar registros
        count_query = f"SELECT COUNT(*) as total FROM {table_name}"
        count = pd.read_sql_query(count_query, conn)['total'][0]
        print(f"Total de registros: {count:,}")

## 4. Carregamento e Visualização Inicial dos Dados

In [ ]:
# Selecionar a tabela principal para análise
# AJUSTE ESTA VARIÁVEL COM O NOME DA SUA TABELA
TABLE_NAME = tables_df['name'][0] if not tables_df.empty else "sua_tabela"

print(f"Analisando a tabela: {TABLE_NAME}")
print("\nSe desejar analisar outra tabela, altere a variável TABLE_NAME acima.")

In [ ]:
# Carregar dados da tabela
if conn:
    query = f"SELECT * FROM {TABLE_NAME}"
    df = pd.read_sql_query(query, conn)
    
    print(f"Dados carregados: {df.shape[0]:,} linhas x {df.shape[1]} colunas")
    print("\nPrimeiras linhas:")
    display(df.head(10))

In [ ]:
# Informações gerais do dataset
print("INFORMAÇÕES GERAIS DO DATASET")
print("="*80)
df.info()

## 5. Análise de Qualidade dos Dados

In [ ]:
# Análise de valores ausentes
def analyze_missing_values(df):
    missing = pd.DataFrame({
        'Coluna': df.columns,
        'Total_Missing': df.isnull().sum(),
        'Percentual_Missing': (df.isnull().sum() / len(df) * 100).round(2),
        'Tipo_Dado': df.dtypes
    })
    missing = missing[missing['Total_Missing'] > 0].sort_values('Total_Missing', ascending=False)
    return missing

missing_analysis = analyze_missing_values(df)

if not missing_analysis.empty:
    print("\n⚠️ VALORES AUSENTES DETECTADOS")
    print("="*80)
    display(missing_analysis)
    
    # Visualizar valores ausentes
    plt.figure(figsize=(12, 6))
    plt.barh(missing_analysis['Coluna'], missing_analysis['Percentual_Missing'])
    plt.xlabel('Percentual de Valores Ausentes (%)')
    plt.title('Análise de Dados Ausentes por Coluna')
    plt.tight_layout()
    plt.show()
else:
    print("\n✅ Nenhum valor ausente detectado!")

In [ ]:
# Análise de duplicatas
duplicates = df.duplicated().sum()
print(f"\nTotal de registros duplicados: {duplicates:,} ({duplicates/len(df)*100:.2f}%)")

if duplicates > 0:
    print("\nExemplo de registros duplicados:")
    display(df[df.duplicated(keep=False)].head())

## 6. Classificação Automática de Variáveis

In [ ]:
# Identificar tipos de variáveis
def classify_variables(df, text_threshold=50):
    numeric_vars = df.select_dtypes(include=[np.number]).columns.tolist()
    
    categorical_vars = []
    text_vars = []
    
    for col in df.select_dtypes(include=['object']).columns:
        unique_ratio = df[col].nunique() / len(df)
        avg_length = df[col].astype(str).str.len().mean()
        
        if avg_length > text_threshold or unique_ratio > 0.5:
            text_vars.append(col)
        else:
            categorical_vars.append(col)
    
    return {
        'numericas': numeric_vars,
        'categoricas': categorical_vars,
        'texto': text_vars
    }

var_types = classify_variables(df)

print("\n📊 CLASSIFICAÇÃO DE VARIÁVEIS")
print("="*80)
print(f"\n✓ Variáveis Numéricas ({len(var_types['numericas'])}):", var_types['numericas'])
print(f"\n✓ Variáveis Categóricas ({len(var_types['categoricas'])}):", var_types['categoricas'])
print(f"\n✓ Campos de Texto ({len(var_types['texto'])}):", var_types['texto'])

## 7. Análise de Variáveis Numéricas

In [ ]:
# Estatísticas descritivas das variáveis numéricas
if var_types['numericas']:
    print("\n📈 ESTATÍSTICAS DESCRITIVAS - VARIÁVEIS NUMÉRICAS")
    print("="*80)
    display(df[var_types['numericas']].describe().T)

In [ ]:
# Distribuições das variáveis numéricas
if var_types['numericas']:
    n_cols = min(3, len(var_types['numericas']))
    n_rows = (len(var_types['numericas']) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if len(var_types['numericas']) > 1 else [axes]
    
    for idx, col in enumerate(var_types['numericas']):
        df[col].hist(bins=30, ax=axes[idx], edgecolor='black', alpha=0.7)
        axes[idx].set_title(f'Distribuição: {col}')
        axes[idx].set_xlabel(col)
        axes[idx].set_ylabel('Frequência')
    
    # Remover subplots vazios
    for idx in range(len(var_types['numericas']), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Boxplots para detectar outliers
if var_types['numericas']:
    n_cols = min(3, len(var_types['numericas']))
    n_rows = (len(var_types['numericas']) + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5*n_rows))
    axes = axes.flatten() if len(var_types['numericas']) > 1 else [axes]
    
    for idx, col in enumerate(var_types['numericas']):
        df.boxplot(column=col, ax=axes[idx])
        axes[idx].set_title(f'Boxplot: {col}')
        axes[idx].set_ylabel(col)
    
    # Remover subplots vazios
    for idx in range(len(var_types['numericas']), len(axes)):
        fig.delaxes(axes[idx])
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Matriz de correlação
if len(var_types['numericas']) > 1:
    plt.figure(figsize=(12, 10))
    correlation_matrix = df[var_types['numericas']].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, linewidths=1, fmt='.2f')
    plt.title('Matriz de Correlação - Variáveis Numéricas')
    plt.tight_layout()
    plt.show()
    
    # Correlações mais fortes
    print("\n🔗 CORRELAÇÕES MAIS FORTES (|r| > 0.5):")
    print("="*80)
    corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            if abs(correlation_matrix.iloc[i, j]) > 0.5:
                corr_pairs.append({
                    'Var1': correlation_matrix.columns[i],
                    'Var2': correlation_matrix.columns[j],
                    'Correlação': correlation_matrix.iloc[i, j]
                })
    
    if corr_pairs:
        corr_df = pd.DataFrame(corr_pairs).sort_values('Correlação', ascending=False, key=abs)
        display(corr_df)
    else:
        print("Nenhuma correlação forte detectada.")

## 8. Análise de Variáveis Categóricas

In [ ]:
# Análise de cardinalidade das variáveis categóricas
if var_types['categoricas']:
    print("\n📋 ANÁLISE DE VARIÁVEIS CATEGÓRICAS")
    print("="*80)
    
    categorical_summary = []
    for col in var_types['categoricas']:
        categorical_summary.append({
            'Variável': col,
            'Valores_Únicos': df[col].nunique(),
            'Valor_Mais_Frequente': df[col].mode()[0] if not df[col].mode().empty else None,
            'Frequência_Máxima': df[col].value_counts().iloc[0] if len(df[col]) > 0 else 0,
            'Percentual_Máximo': f"{df[col].value_counts(normalize=True).iloc[0]*100:.2f}%" if len(df[col]) > 0 else "0%"
        })
    
    cat_summary_df = pd.DataFrame(categorical_summary)
    display(cat_summary_df)

In [ ]:
# Distribuição de frequências das variáveis categóricas
if var_types['categoricas']:
    for col in var_types['categoricas']:
        print(f"\n📊 Distribuição: {col}")
        print("-"*80)
        
        value_counts = df[col].value_counts()
        
        # Mostrar top 10 valores
        print(f"\nTop 10 valores mais frequentes:")
        display(pd.DataFrame({
            'Valor': value_counts.head(10).index,
            'Frequência': value_counts.head(10).values,
            'Percentual': (value_counts.head(10).values / len(df) * 100).round(2)
        }))
        
        # Gráfico de barras (limitado a top 15 para legibilidade)
        if len(value_counts) <= 15:
            plt.figure(figsize=(12, 6))
            value_counts.plot(kind='bar', edgecolor='black', alpha=0.7)
            plt.title(f'Distribuição de Frequências: {col}')
            plt.xlabel(col)
            plt.ylabel('Frequência')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()
        else:
            plt.figure(figsize=(12, 6))
            value_counts.head(15).plot(kind='bar', edgecolor='black', alpha=0.7)
            plt.title(f'Top 15 Valores - {col}')
            plt.xlabel(col)
            plt.ylabel('Frequência')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.show()

## 9. Análise de Campos de Texto

In [ ]:
# Análise de campos de texto
if var_types['texto']:
    print("\n📝 ANÁLISE DE CAMPOS DE TEXTO")
    print("="*80)
    
    text_summary = []
    for col in var_types['texto']:
        text_lengths = df[col].astype(str).str.len()
        text_summary.append({
            'Campo': col,
            'Valores_Únicos': df[col].nunique(),
            'Comprimento_Médio': text_lengths.mean().round(2),
            'Comprimento_Min': text_lengths.min(),
            'Comprimento_Max': text_lengths.max(),
            'Comprimento_Mediana': text_lengths.median()
        })
    
    text_summary_df = pd.DataFrame(text_summary)
    display(text_summary_df)

In [ ]:
# Distribuição de comprimento de texto
if var_types['texto']:
    for col in var_types['texto']:
        print(f"\n📊 Distribuição de Comprimento: {col}")
        print("-"*80)
        
        text_lengths = df[col].astype(str).str.len()
        
        plt.figure(figsize=(12, 5))
        
        plt.subplot(1, 2, 1)
        plt.hist(text_lengths, bins=30, edgecolor='black', alpha=0.7)
        plt.xlabel('Comprimento do Texto')
        plt.ylabel('Frequência')
        plt.title(f'Histograma de Comprimento: {col}')
        
        plt.subplot(1, 2, 2)
        plt.boxplot(text_lengths)
        plt.ylabel('Comprimento do Texto')
        plt.title(f'Boxplot de Comprimento: {col}')
        
        plt.tight_layout()
        plt.show()
        
        # Exemplos de textos
        print(f"\nExemplos de textos (primeiros 5 valores únicos):")
        for idx, text in enumerate(df[col].dropna().unique()[:5], 1):
            preview = str(text)[:100] + "..." if len(str(text)) > 100 else str(text)
            print(f"{idx}. {preview}")

## 10. Análise de Palavras mais Frequentes (Campos de Texto)

In [ ]:
# Análise de palavras mais frequentes
from collections import Counter
import re

if var_types['texto']:
    for col in var_types['texto']:
        print(f"\n🔤 Palavras Mais Frequentes: {col}")
        print("-"*80)
        
        # Concatenar todos os textos
        all_text = ' '.join(df[col].astype(str).values)
        
        # Extrair palavras (apenas letras, mínimo 3 caracteres)
        words = re.findall(r'\b[a-záàâãéèêíïóôõöúçñ]{3,}\b', all_text.lower())
        
        # Contar palavras
        word_counts = Counter(words)
        
        # Top 20 palavras
        top_words = pd.DataFrame(word_counts.most_common(20), 
                                columns=['Palavra', 'Frequência'])
        
        display(top_words)
        
        # Gráfico de barras
        plt.figure(figsize=(12, 6))
        plt.barh(top_words['Palavra'][::-1], top_words['Frequência'][::-1], alpha=0.7)
        plt.xlabel('Frequência')
        plt.title(f'Top 20 Palavras Mais Frequentes: {col}')
        plt.tight_layout()
        plt.show()

## 11. Relatório de Preparação para Redes Neurais

In [ ]:
# Gerar relatório de preparação
print("\n" + "="*80)
print("RELATÓRIO DE PREPARAÇÃO PARA TREINAMENTO DE REDES NEURAIS")
print("="*80)

print(f"\n📊 RESUMO DO DATASET")
print(f"  • Total de registros: {len(df):,}")
print(f"  • Total de features: {df.shape[1]}")
print(f"  • Duplicatas: {duplicates:,}")

print(f"\n🔢 VARIÁVEIS NUMÉRICAS ({len(var_types['numericas'])})")
if var_types['numericas']:
    for var in var_types['numericas']:
        print(f"  • {var}")
    print("\n  Recomendações:")
    print("    - Normalizar/Padronizar antes do treinamento")
    print("    - Verificar outliers e decidir tratamento")
    print("    - Analisar correlações para feature engineering")

print(f"\n📋 VARIÁVEIS CATEGÓRICAS ({len(var_types['categoricas'])})")
if var_types['categoricas']:
    for var in var_types['categoricas']:
        n_unique = df[var].nunique()
        print(f"  • {var} ({n_unique} categorias únicas)")
    print("\n  Recomendações:")
    print("    - One-Hot Encoding para baixa cardinalidade (<10 categorias)")
    print("    - Label Encoding ou Embedding para alta cardinalidade")
    print("    - Considerar Target Encoding se houver variável alvo")

print(f"\n📝 CAMPOS DE TEXTO ({len(var_types['texto'])})")
if var_types['texto']:
    for var in var_types['texto']:
        print(f"  • {var}")
    print("\n  Recomendações:")
    print("    - TF-IDF ou Word Embeddings (Word2Vec, GloVe)")
    print("    - BERT/Transformers para contexto semântico")
    print("    - Pré-processamento: remoção de stopwords, stemming/lemmatization")

print(f"\n⚠️ PROBLEMAS IDENTIFICADOS")
issues = []
if not missing_analysis.empty:
    issues.append(f"  • Valores ausentes em {len(missing_analysis)} colunas")
if duplicates > 0:
    issues.append(f"  • {duplicates:,} registros duplicados")

if issues:
    for issue in issues:
        print(issue)
else:
    print("  ✅ Nenhum problema crítico detectado")

print(f"\n🎯 PRÓXIMOS PASSOS")
print("  1. Tratamento de valores ausentes (imputação ou remoção)")
print("  2. Codificação de variáveis categóricas")
print("  3. Processamento de texto (tokenização, vetorização)")
print("  4. Normalização de features numéricas")
print("  5. Split train/validation/test")
print("  6. Feature engineering se necessário")
print("  7. Definição da arquitetura da rede neural")

## 12. Exportar Dados para Análise Posterior

In [ ]:
# Salvar dados em CSV para análise posterior
output_path = r"c:\Users\bruno.silva\Codes\cnn-classificador-peticoes\dados_exportados.csv"
df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"\n✅ Dados exportados com sucesso para: {output_path}")

In [ ]:
# Salvar relatório de tipos de variáveis
import json

report = {
    'dataset_info': {
        'total_rows': len(df),
        'total_columns': df.shape[1],
        'duplicates': int(duplicates)
    },
    'variable_types': var_types,
    'missing_values': missing_analysis.to_dict('records') if not missing_analysis.empty else []
}

report_path = r"c:\Users\bruno.silva\Codes\cnn-classificador-peticoes\analise_relatorio.json"
with open(report_path, 'w', encoding='utf-8') as f:
    json.dump(report, f, indent=2, ensure_ascii=False)

print(f"\n✅ Relatório salvo em: {report_path}")

## 13. Fechar Conexão com o Banco de Dados

In [ ]:
# Fechar conexão
if conn:
    conn.close()
    print("\n✅ Conexão com o banco de dados fechada com sucesso!")

---

## Conclusão

Este notebook realizou uma análise exploratória completa dos dados SQLite3, incluindo:

- ✅ Conexão e exploração da estrutura do banco
- ✅ Análise de qualidade dos dados (valores ausentes, duplicatas)
- ✅ Classificação automática de variáveis (numéricas, categóricas, texto)
- ✅ Análise estatística detalhada de cada tipo de variável
- ✅ Visualizações para melhor compreensão dos dados
- ✅ Recomendações para pré-processamento e treinamento de redes neurais

Os dados estão prontos para as próximas etapas de preparação e modelagem!